# 1. Import necessary Libraries

In [1]:
!pip install ortools
!pip install pandas

In [1]:
import pandas as pd

In [2]:
# open nominal files
# example of one nominal file
with open('./PIE_SXS10_data/nominal/scenario_10SAT_nominal1.txt') as f:
    lines = f.readlines()

In [3]:
list_sat = lines[1].strip().split('\t')
list_sat # list of sat

['SAT15',
 'SAT12',
 'SAT21',
 'SAT17',
 'SAT07',
 'SAT09',
 'SAT22',
 'SAT11',
 'SAT18',
 'SAT03']

# 2. Read and Parse requirements for each task

In [4]:
# requirements for each task
data_i = []
for i in range(3,len(lines)):
    data_i.append(lines[i].strip().split())

In [5]:
data_i

[['TASK0', 'SAT15', '8', '1851', '0', '1207749', '1', '-1', '25200', '43200'],
 ['TASK4', 'SAT12', '8', '1980', '0', '1207620', '1', '-1', '25200', '43200'],
 ['TASK8', 'SAT21', '8', '1810', '0', '1207790', '1', '-1', '25200', '43200'],
 ['TASK12', 'SAT17', '8', '2227', '0', '1207373', '1', '-1', '25200', '43200'],
 ['TASK16', 'SAT07', '8', '1983', '0', '1207617', '1', '-1', '25200', '43200'],
 ['TASK20', 'SAT09', '8', '2185', '0', '1207415', '1', '-1', '25200', '43200'],
 ['TASK24', 'SAT22', '8', '2328', '0', '1207272', '1', '-1', '25200', '43200'],
 ['TASK28', 'SAT11', '8', '2275', '0', '1207325', '1', '-1', '25200', '43200'],
 ['TASK32', 'SAT18', '8', '1934', '0', '1207666', '1', '-1', '25200', '43200'],
 ['TASK36', 'SAT03', '8', '1999', '0', '1207601', '1', '-1', '25200', '43200']]

In [6]:
# header for tasks' requirements
header = ["Task number","Satellite","Priority","Duration","Earliest","Latest","Repetitive","Number occ","Min time lag","Max time lag"]

In [7]:
header

['Task number',
 'Satellite',
 'Priority',
 'Duration',
 'Earliest',
 'Latest',
 'Repetitive',
 'Number occ',
 'Min time lag',
 'Max time lag']

In [8]:
data_df = pd.DataFrame(data_i,columns=header) # data frame for nominal data

In [9]:
data_df

,Task number,Satellite,Priority,Duration,Earliest,Latest,Repetitive,Number occ,Min time lag,Max time lag
0,TASK0,SAT15,8,1851,0,1207749,1,-1,25200,43200
1,TASK4,SAT12,8,1980,0,1207620,1,-1,25200,43200
2,TASK8,SAT21,8,1810,0,1207790,1,-1,25200,43200
3,TASK12,SAT17,8,2227,0,1207373,1,-1,25200,43200
4,TASK16,SAT07,8,1983,0,1207617,1,-1,25200,43200
5,TASK20,SAT09,8,2185,0,1207415,1,-1,25200,43200
6,TASK24,SAT22,8,2328,0,1207272,1,-1,25200,43200
7,TASK28,SAT11,8,2275,0,1207325,1,-1,25200,43200
8,TASK32,SAT18,8,1934,0,1207666,1,-1,25200,43200
9,TASK36,SAT03,8,1999,0,1207601,1,-1,25200,43200


# 3. Read and Parse Visibilities

In [10]:
# open visibility files
with open('./PIE_SXS10_data/visibilities.txt') as f:
    visibs = f.readlines()

visibs

['         Sat           Ant         Start           End\n',
 '       SAT01          ANT7             0          8994\n',
 '       SAT01          ANT9             0          8994\n',
 '       SAT01          ANT7        907200        916194\n',
 '       SAT01          ANT9        907200        916194\n',
 '       SAT01          ANT7         21266         48919\n',
 '       SAT01          ANT9         21266         48919\n',
 '       SAT01          ANT7        928466        956119\n',
 '       SAT01          ANT9        928466        956119\n',
 '       SAT01          ANT7        127519        166841\n',
 '       SAT01          ANT9        127519        166841\n',
 '       SAT01          ANT7       1034719       1074041\n',
 '       SAT01          ANT9       1034719       1074041\n',
 '       SAT01          ANT7        246675        268290\n',
 '       SAT01          ANT9        246675        268290\n',
 '       SAT01          ANT7       1153875       1175490\n',
 '       SAT01          

In [11]:
header_visibs = []
data_visibs = []
for i in range(len(visibs)):
    if i == 0:
        header_visibs = visibs[i].strip().split()
    else:
        data_visibs.append(visibs[i].strip().split())

In [12]:
header_visibs

['Sat', 'Ant', 'Start', 'End']

In [13]:
data_visib_df = pd.DataFrame(data_visibs,columns=header_visibs)
data_visib_df

,Sat,Ant,Start,End
0,SAT01,ANT7,0,8994
1,SAT01,ANT9,0,8994
2,SAT01,ANT7,907200,916194
3,SAT01,ANT9,907200,916194
4,SAT01,ANT7,21266,48919
...,...,...,...,...
12953,SAT47,ANT8,756375,766024
12954,SAT47,ANT6,781174,812509
12955,SAT47,ANT8,781174,812509
12956,SAT47,ANT6,888189,907200


# 4. CP Model and Intervals

In [14]:
import collections
from ortools.sat.python import cp_model

In [15]:
model = cp_model.CpModel()

# 5. Classes and Objects

NewIntervalVar

def NewIntervalVar(self, start, size, end, name)

Creates an interval variable from start, size, and end.

An interval variable is a constraint, that is itself used in other constraints like NoOverlap.

Internally, it ensures that start + size == end.

In [16]:
class Task:
    def __init__(self, i, sat_id, priority, earliest, lastest, interval, sat, ant_id):
        self.i = i # id de tash
        # self.sid = sat_id # id de sat
        self.priority = priority
        self.earliest = earliest
        self.lastest = lastest
        self.interval = interval # model.NewIntervalVar(0,0,0,'task'+i.tostring) # without repetation
        self.sat = sat
        self.ant_id = ant_id
        
class Sat:
    def __init__(self, i, visib, tasks):
        self.i = i
        self.visib = visib # visib est une liste de antenne et intervals
        self.tasks = tasks

class Ant:
    def __init__(sefl, i, intervals):
        self.i = i
        self.intervals = intervals # liste d'intervalles de fonctionnement

In [17]:
data_visib_df

,Sat,Ant,Start,End
0,SAT01,ANT7,0,8994
1,SAT01,ANT9,0,8994
2,SAT01,ANT7,907200,916194
3,SAT01,ANT9,907200,916194
4,SAT01,ANT7,21266,48919
...,...,...,...,...
12953,SAT47,ANT8,756375,766024
12954,SAT47,ANT6,781174,812509
12955,SAT47,ANT8,781174,812509
12956,SAT47,ANT6,888189,907200


In [18]:
# construct objects of Ant
d_ants = {}
all_sats = []
for i in range(len(data_visib_df['Ant'])) :
    name = data_visib_df['Ant'][i]
    ID = int(name.strip('ANT'))
    start = int(data_visib_df['Start'][i])
    end = int(data_visib_df['End'][i])
    if ID in d_ants:
        count = len(d_ants[ID])
        d_ants[ID].append(model.NewIntervalVar(start,end-start,end,str(count+1)))
    else:
        d_ants[ID] = []
        d_ants[ID].append(model.NewIntervalVar(start,end-start,end,str(1)))
        
def get_dict_ant_for_sat(data_visib):
    d_ants = {}
    for i in range(len(data_visib['Ant'])) :
        name = data_visib['Ant'][i]
        ID = int(name.strip('ANT'))
        start = int(data_visib['Start'][i])
        end = int(data_visib['End'][i])
        if ID in d_ants:
            count = len(d_ants[ID])
            d_ants[ID].append(model.NewIntervalVar(start,end-start,end,str(count+1)))
        else:
            d_ants[ID] = []
            d_ants[ID].append(model.NewIntervalVar(start,end-start,end,str(1)))
    return d_ants
        

In [19]:
d_ants.keys()

dict_keys([7, 9, 5, 10, 1, 2, 3, 11, 6, 8, 4, 12])

In [20]:
data_df

,Task number,Satellite,Priority,Duration,Earliest,Latest,Repetitive,Number occ,Min time lag,Max time lag
0,TASK0,SAT15,8,1851,0,1207749,1,-1,25200,43200
1,TASK4,SAT12,8,1980,0,1207620,1,-1,25200,43200
2,TASK8,SAT21,8,1810,0,1207790,1,-1,25200,43200
3,TASK12,SAT17,8,2227,0,1207373,1,-1,25200,43200
4,TASK16,SAT07,8,1983,0,1207617,1,-1,25200,43200
5,TASK20,SAT09,8,2185,0,1207415,1,-1,25200,43200
6,TASK24,SAT22,8,2328,0,1207272,1,-1,25200,43200
7,TASK28,SAT11,8,2275,0,1207325,1,-1,25200,43200
8,TASK32,SAT18,8,1934,0,1207666,1,-1,25200,43200
9,TASK36,SAT03,8,1999,0,1207601,1,-1,25200,43200


In [21]:
# create sat objects
# consider only one file
d_sats = {}
for sat in list_sat:
    name = data_df['Satellite'][i]
    data_visib = data_visibility_df[data_visibility_df['Sat']==name]
    d_ant_tmp = get_dict_ant_for_sat(data_visib)
    ID = int(name.strip('SAT'))
    priority = data_df['Priority'][i]
    eariest = int(data_df['Eariest'][i])
    latest = int(data_visib_df['Latest'][i])

KeyError: 12957

NameError: name 'd_ant_tmp' is not defined

In [25]:
# int(data_visib_df["Sat"][0].strip('SAT'))

# list_ant = []
# list_sat = []
# dict_sat = {}
# for i in range(len(data_visib_df["Sat"])):
#     sat_i = Sat()

TypeError: __init__() missing 3 required positional arguments: 'i', 'visib', and 'tasks'

In [ ]:
# # Import the libraries
# from ortools.sat.python import cp_model

# # Declare the model
# model = cp_model.CpModel()

# # Create the variables de Interval


# # Create the constraint
# model.Add(x != y)

# # Call the solver
# solver = cp_model.CpSolver()
# status = solver.Solve(model)

0        ANT7
1        ANT9
2        ANT7
3        ANT9
4        ANT7
         ... 
12953    ANT8
12954    ANT6
12955    ANT8
12956    ANT6
12957    ANT8
Name: Ant, Length: 12958, dtype: object